# Assigment week 2

## Exercise 1

Your collaborator has performed an experiment and has measured the spike counts of two neurons when presenting 10 different stimuli. Each stimulus was presented 20 times. The data is saved in the file `spike_counts.npz` in two arrays, one for each cell (keys `spikes_neuron1` and `spikes_neuron2`), in which the rows are the number of stimuli presented and the columns are the 20 trials. 

First of all, plot these arrays - can you visually see which neuron conveys more information about the stimulus identity? 

By computing the mutual information between the spike count of each cell and the stimuli presented, determine the answer in a quantitative fashion. In doing so, bin the spike counts into bins: 0-4, 5-9, 10-14 etc and use logarithms to base 2 in the definitions. When computing the joint probability density function, do not smooth this in any way.

**Return (to 3 decimal places):**

  1. the mutual information between neuron 1 and the stimulus
  2. the mutual information between neuron 2 and the stimulus
  3. the maximum mutual information that a neuron could convey about the stimulus
  
 
---

Now, bin the counts into bins of 10 (0-9, 10-19, 20-29 etc.).

**Return (to 3 decimal places):**

  4. the mutual information between neuron 1 and the stimulus with bins of 10 spikes
  5. the mutual information between neuron 2 and the stimulus with bins of 10 spikes

Do you understand why this is the case?

If you now take the bin size to be equal to 1, you find that the mutual information between neuron 1 and the stimulus is 1.897, whereas that between neuron 2 and the stimulus is 1.810. Do you understand why?

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive/Colab Notebooks
spcounts= np.load('spike_counts.npz')
spcounts=dict(zip(('spikes_neuron1','spikes_neuron2','spikes_unknown_stim'), (spcounts[k] for k in spcounts)))

#print(spcounts['spikes_neuron1'])
#calculate the conditional probability function
def x_giveny(data,bins):
 px_giveny_stim=[]
 for i in range(data.shape[0]):
   #print(i)
   #print(spcounts['spikes_neuron1'][i])
   hist, edges=np.histogram(data[i], np.arange(0,np.amax(data[i]),bins))
   px_giveny_stim.append(hist/20)
   #print(hist/20)
 px_giveny= pd.DataFrame(px_giveny_stim,dtype=float)
 px_giveny=px_giveny.fillna(0).values
 return px_giveny
#calculate the marginal probability of x
def px(px_giveny,py):
  px= np.sum(py*px_giveny, axis=0)
  return px


#calculate the mutual information
def mutual_info(px, py, px_giveny):
  sum=0
  for i in range(px_giveny.shape[0]):
    for j in range(px_giveny.shape[1]):
      if px_giveny[i][j] != 0:
        #sum = sum + math.log2(px_giveny[i][j]/px[j])
        sum= sum+ py*px_giveny[i][j]*math.log2(px_giveny[i][j]/px[j])
  return sum

#bins of length 5
py = 0.1  
px_giveny1 = x_giveny(spcounts['spikes_neuron1'],5)
mutualinfo_n1 = round(mutual_info(px(px_giveny1,py),py,px_giveny1),3)
print('The mutual information between neuron 1 and the stimulus is '+ str(mutualinfo_n1))
px_giveny2 = x_giveny(spcounts['spikes_neuron2'],5)
mutualinfo_n2 = round(mutual_info(px(px_giveny2,py),py,px_giveny2),3)
print('The mutual information between neuron 2 and the stimulus is '+ str(mutualinfo_n2))

#bins of length 10
py = 0.1  
px_giveny1 = x_giveny(spcounts['spikes_neuron1'],10)
mutualinfo_n1 = round(mutual_info(px(px_giveny1,py),py,px_giveny1),3)
print('The mutual information between neuron 1 and the stimulus is '+ str(mutualinfo_n1))
px_giveny2 = x_giveny(spcounts['spikes_neuron2'],10)
mutualinfo_n2 = round(mutual_info(px(px_giveny2,py),py,px_giveny2),3)
print('The mutual information between neuron 2 and the stimulus is '+ str(mutualinfo_n2))

#bins of length 1
py = 0.1  
px_giveny1 = x_giveny(spcounts['spikes_neuron1'],1)
mutualinfo_n1 = round(mutual_info(px(px_giveny1,py),py,px_giveny1),3)
print('The mutual information between neuron 1 and the stimulus is '+ str(mutualinfo_n1))
px_giveny2 = x_giveny(spcounts['spikes_neuron2'],1)
mutualinfo_n2 = round(mutual_info(px(px_giveny2,py),py,px_giveny2),3)
print('The mutual information between neuron 2 and the stimulus is '+ str(mutualinfo_n2))




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks'
/content/drive/My Drive/Colab Notebooks
The mutual information between neuron 1 and the stimulus is 0.848
The mutual information between neuron 2 and the stimulus is 1.124
The mutual information between neuron 1 and the stimulus is 0.576
The mutual information between neuron 2 and the stimulus is 1.015
The mutual information between neuron 1 and the stimulus is 1.831
The mutual information between neuron 2 and the stimulus is 1.774


# Exercise 2

Your collaborator also tells you that there are 20 more trials of neuron 1, but that it is unclear whether they belong to stimulus 7 or stimulus 8.

You decide to help your collaborator out. Assume that the spike number from neuron 1 to all stimuli is a Gaussian function, where the mean depends on the stimulus identity but the variance is fixed. First, use maximum likelihood estimation to estimate the mean firing rates to stimuli 7 and 8 (**start counting with 1**, so they will be entries `spikes_neuronn[6, :]` and `spikes_neuronn[7, :]` of the matrix).


**Return (to 3 decimal places):**

  6. the estimate for the mean number of spikes that neuron 1 fires to stimulus 7
  7. the estimate for the mean number of spikes that neuron 1 fires to stimulus 8
  8. the estimate for the variance of the distributions (using all 200 responses to all stimuli)
  
---

Then, load the data coming from the unknown stimulus. You will find the data under the key `spikes_unknown_stim` of the `spike_counts.npz` file.
Use the log-likelihood ratio function to decide what stimulus is your best guess. What is the probability that you are wrong?

**Return:**

  9. Your guess for the stimulus (7 or 8)
  10. The probability that you are wrong (up to 3 decimal places)  



In [27]:
import scipy.stats

mean7 = np.mean(spcounts['spikes_neuron1'][6,:])
mean8 = np.mean(spcounts['spikes_neuron1'][7,:])
var_stimuli = np.var(spcounts['spikes_neuron1'],axis=1)
var= np.mean(var_stimuli)
#print(var)
print(var_stimuli)
print(mean7)
print(mean8)

logR = (mean8-mean7)*np.sum(2*spcounts['spikes_unknown_stim']-mean7-mean8)/(2*var)
print(logR)

if logR>0:
  print('The best guess is stimulus 8')
else:
  print('The best guess is stimulus 7')

Rmean = 20*(mean8-mean7)**2/(2*var) 
Rvar = 20*(mean8-mean7)**2/var   
P_wrong = round((1-scipy.stats.norm.cdf(logR, loc= -Rmean, scale=math.sqrt(Rvar))),3)
print('The probability of the guess being wrong is '+ str(P_wrong))


[198.26   167.4475 337.5875 363.4875 393.2    270.16   269.51   202.8275
 226.91   555.34  ]
34.3
45.15
1.5449471141443305
The best guess is stimulus 8
The probability of the guess being wrong is 0.025
